In [19]:
import time
import random
from typing import Dict, Any, List
from contextlib import contextmanager

import opik
from opik.integrations.openai import track_openai


 # Dynamic Tracing Control Cookbook



 This cookbook demonstrates how to use Opik's dynamic tracing control features

 to optimize performance and implement flexible tracing strategies in production.



 ## What You'll Learn:

 - Enable/disable tracing at runtime without code changes

 - Implement conditional tracing based on user attributes

 - Create sampling strategies for high-throughput systems

 - Measure and optimize tracing performance impact

 - Control integration tracking dynamically



 ## Prerequisites:

 ```bash

 pip install opik

 ```

 ## Setup and Imports



 First, let's import the necessary libraries and set up our environment.

In [20]:
print(f"Opik version: {opik.__version__}")
print(f"Initial tracing state: {opik.is_tracing_active()}")


Opik version: 1.8.7
Initial tracing state: False


 ## 1. Basic Runtime Control



 The simplest use case is toggling tracing on and off during runtime.

In [21]:
print(f"Current tracing state: {opik.is_tracing_active()}")

opik.set_tracing_active(False)
print(f"After disabling: {opik.is_tracing_active()}")

opik.set_tracing_active(True)
print(f"After enabling: {opik.is_tracing_active()}")


Current tracing state: False
After disabling: False
After enabling: True


 ## 2. Context-Aware Tracing



 Create a context manager for temporary tracing control.

In [22]:
@contextmanager
def tracing_enabled(enabled: bool):
    """Context manager for temporary tracing control."""
    original_state = opik.is_tracing_active()
    try:
        opik.set_tracing_active(enabled)
        yield
    finally:
        opik.set_tracing_active(original_state)

print(f"Before context: {opik.is_tracing_active()}")

with tracing_enabled(False):
    print(f"Inside context (disabled): {opik.is_tracing_active()}")
    # ny traced functions here won't create spans

print(f"After context: {opik.is_tracing_active()}")


Before context: True
Inside context (disabled): False
After context: True


 ## 3. Conditional Tracing Strategies



 Implement different strategies for when to enable tracing.

In [23]:
class TracingStrategy:
    """Base class for tracing strategies."""

    def should_trace(self, **kwargs) -> bool:
        """Determine if tracing should be enabled for this request."""
        raise NotImplementedError


class UserTierStrategy(TracingStrategy):
    """Trace only premium users."""

    def __init__(self, premium_tiers: List[str] = None):
        self.premium_tiers = premium_tiers or ["premium", "enterprise"]

    def should_trace(self, user_tier: str = None, **kwargs) -> bool:
        return user_tier in self.premium_tiers


class SamplingStrategy(TracingStrategy):
    """Trace a percentage of requests."""

    def __init__(self, sample_rate: float = 0.1):
        self.sample_rate = sample_rate

    def should_trace(self, **kwargs) -> bool:
        return random.random() < self.sample_rate


class DebugModeStrategy(TracingStrategy):
    """Trace when in debug mode or for specific users."""

    def __init__(self, debug_users: List[str] = None):
        self.debug_users = debug_users or []
        self.debug_mode = False

    def should_trace(self, user_id: str = None, **kwargs) -> bool:
        return self.debug_mode or (user_id in self.debug_users)

    def enable_debug(self):
        self.debug_mode = True

    def disable_debug(self):
        self.debug_mode = False

strategies = {
    "premium_only": UserTierStrategy(),
    "10_percent_sample": SamplingStrategy(0.1),
    "debug_mode": DebugModeStrategy(["debug_user_1", "debug_user_2"]),
}


 ## 4. Smart Request Handler



 Create a request handler that uses tracing strategies.

In [24]:
@opik.track(name="user_query")
def handle_user_query(query: str, user_id: str, user_tier: str) -> Dict[str, Any]:
    """Simulate handling a user query with LLM processing."""
    # Simulate some processing time
    time.sleep(0.01)
    
    # Mock LLM response based on user tier
    if user_tier in ["premium", "enterprise"]:
        response = f"Premium response to: {query}"
        tokens_used = random.randint(150, 300)
    else:
        response = f"Basic response to: {query}"
        tokens_used = random.randint(50, 100)
    
    return {
        "query": query,
        "response": response,
        "user_id": user_id,
        "user_tier": user_tier,
        "tokens_used": tokens_used,
        "processing_time_ms": random.randint(100, 500)
    }


@opik.track(name="data_enrichment")
def enrich_data(data: Dict[str, Any]) -> Dict[str, Any]:
    """Enrich response data with additional metadata."""
    # Simulate data enrichment processing
    time.sleep(0.005)
    
    enriched = data.copy()
    enriched.update({
        "timestamp": time.time(),
        "enrichment_version": "v1.2",
        "quality_score": random.uniform(0.7, 1.0),
        "confidence": random.uniform(0.8, 0.95)
    })
    
    return enriched


class SmartRequestHandler:
    """Request handler with configurable tracing strategy."""

    def __init__(self, strategy: TracingStrategy):
        self.strategy = strategy
        self.request_count = 0
        self.traced_count = 0

    def handle_request(
        self, query: str, user_id: str, user_tier: str
    ) -> Dict[str, Any]:
        """Handle request with conditional tracing."""
        self.request_count += 1

        should_trace = self.strategy.should_trace(
            user_id=user_id, user_tier=user_tier, query=query
        )

        opik.set_tracing_active(should_trace)

        if should_trace:
            self.traced_count += 1

        result = handle_user_query(query, user_id, user_tier)
        enriched = enrich_data(result)

        return {
            **enriched,
            "traced": should_trace,
            "request_number": self.request_count,
        }

    def get_stats(self) -> Dict[str, Any]:
        """Get handler statistics."""
        trace_rate = (
            (self.traced_count / self.request_count) if self.request_count > 0 else 0
        )
        return {
            "total_requests": self.request_count,
            "traced_requests": self.traced_count,
            "trace_rate": f"{trace_rate:.1%}",
        }



print("=== Testing Premium-Only Strategy ===")
handler = SmartRequestHandler(UserTierStrategy())

requests = [
    ("What is AI?", "user1", "free"),
    ("Explain ML", "user2", "premium"),
    ("How does it work?", "user3", "free"),
    ("Advanced question", "user4", "enterprise"),
]

for query, user_id, tier in requests:
    result = handler.handle_request(query, user_id, tier)
    print(f"User {user_id} ({tier}): Traced = {result['traced']}")

print(f"Strategy stats: {handler.get_stats()}")


OPIK: Started logging traces to the "Default Project" project at http://localhost:5173/api/v1/session/redirect/projects/?trace_id=01982b4c-d533-7f09-b649-acf3f6e406de&path=aHR0cDovL2xvY2FsaG9zdDo1MTczL2FwaS8=.


=== Testing Premium-Only Strategy ===
User user1 (free): Traced = False
User user2 (premium): Traced = True
User user3 (free): Traced = False
User user4 (enterprise): Traced = True
Strategy stats: {'total_requests': 4, 'traced_requests': 2, 'trace_rate': '50.0%'}
